In [4]:
import streamlit as st
import easyocr
import io
from io import BytesIO
from streamlit_option_menu import option_menu
import mysql.connector
import pandas as pd
import re
from PIL import Image
import numpy as np
import os
from pprint import pprint

In [2]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Nirmal9699',
    database='business_cards')
cursor = mydb.cursor()


create='''create table if not exists bixcard(name varchar(100),
                                        designation varchar(100),
                                        company_name varchar(100),
                                        contact varchar(100),
                                        alternative varchar(100),
                                        email varchar(100),
                                        website varchar(100),
                                        street varchar(100),
                                        city varchar(100),
                                        state varchar(100),
                                        pincode varchar(100),
                                        image_byt longblob)'''
cursor.execute(create)
mydb.commit()


In [19]:
def upload_image(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    details = [res[1] for res in result]

    name = []
    designation = []
    contact = []
    email = []
    website = []
    street = []
    city = []
    state = []
    pincode = []
    company = []

    for detail in details:
        match1 = re.findall('([0-9]+ [A-Z]+ [A-Za-z]+)., ([a-zA-Z]+). ([a-zA-Z]+)', detail)
        match2 = re.findall('([0-9]+ [A-Z]+ [A-Za-z]+)., ([a-zA-Z]+)', detail)
        match3 = re.findall('^[E].+[a-z]', detail)
        match4 = re.findall('([A-Za-z]+) ([0-9]+)', detail)
        match5 = re.findall('([0-9]+ [a-zA-z]+)', detail)
        match6 = re.findall('.com$', detail)
        match7 = re.findall('([0-9]+)', detail)

        if detail == details[0]:
            name.append(detail)
        elif detail == details[1]:
            designation.append(detail)
        elif '-' in detail:
            contact.append(detail)
        elif '@' in detail:
            email.append(detail)
        elif "www " in detail.lower() or "www." in detail.lower():
            website.append(detail)
        elif "WWW" in detail:
            website.append(detail + "." + details[details.index(detail) + 1])
        elif match6:
            pass
        elif match1:
            street.append(match1[0][0])
            city.append(match1[0][1])
            state.append(match1[0][2])
        elif match2:
            street.append(match2[0][0])
            city.append(match2[0][1])
        elif match3:
            city.append(match3[0])
        elif match4:
            state.append(match4[0][0])
            pincode.append(match4[0][1])
        elif match5:
            street.append(match5[0] + ' St,')
        elif match7:
            pincode.append(match7[0])
        else:
            company.append(detail)

    if len(company) > 1:
        comp = company[0] + ' ' + company[1]
    else:
        comp = company[0]

    contact_number = contact[0] if contact else None
    alternative_number = contact[1] if len(contact) > 1 else None

    image_details = {'Name': name[0], 'Designation': designation[0], 'Company_Name': comp,
                     'Contact': contact_number, 'Alternative_Contact': alternative_number, 'Email': email[0],
                     'Website': website[0], 'Street': street[0], 'City': city[0], 'State': state[0],
                     'Pincode': pincode[0],'Image_Path': image_path}

    return image_details

# Example usage for a single image
image_path = r"C:\Guvi\Project\BizCardX_ Extracting Business Card Data with OCR\2.png"
df = pd.DataFrame([upload_image(image_path)])
df

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


,Name,Designation,Company_Name,Contact,Alternative_Contact,Email,Website,Street,City,State,Pincode,Image_Path
0,Amit kumar,CEO & FOUNDER,GLOBAL INSURANCE,123-456-7569,None,hello@global.com,WWW.global.com,"123 global St,",Erode,TamilNadu,600115,C:\Guvi\Project\BizCardX_ Extracting Business ...


In [16]:
df.name

0    Amit kumar
Name: name, dtype: object